In [1]:
service = dict([('title', 'S1 data selection and Crop/GeoTiff conversion'),
                ('abstract', 'Sentinel-1 data selection and crop/GeoTiff conversion'),
                ('id', 's1_crop')])

In [2]:
pa_code = dict([('id', 'pa_code'),
               ('value', 'DO'),
               ('title', 'Protected Area code'),
               ('abstract', 'Protected Area code (one of DO,CA,...)')])

In [3]:
pa_name = dict([('id', 'pa_name'),
               ('value', 'Donana'),
               ('title', 'Protected Area name'),
               ('abstract', 'Protected Area name (one of Donana,...)')])

In [4]:
pa_wkt = dict([('id', 'pa_wkt'),
               ('value', 'POLYGON((-6.79440295589744 37.1191319871777,-6.80827154725944 37.1673031942056,-6.72121114356795 37.2425767108835,-6.71042775152577 37.2930498879682,-6.73405462341917 37.3301447063348,-6.71685147187095 37.3522253716156,-6.67435624435345 37.3534492963332,-6.64461873043217 37.3365633914571,-6.62625336367183 37.3569789562915,-6.59581018720718 37.3506994424118,-6.58125246058411 37.3703051383248,-6.492413660497 37.3590379521576,-6.4716624422663 37.3807359777278,-6.43859705183134 37.368908852933,-6.44398095303713 37.347088527492,-6.42200724935979 37.3329929700694,-6.37035012694059 37.3583637388823,-6.33153540861988 37.3329986280188,-6.25703879672152 37.3486795386779,-6.2123425985781 37.3266608721382,-6.1976208676784 37.283024780795,-6.17366048732846 37.2660742144794,-6.15177710683581 37.2667431439207,-6.14475916204647 37.3055642119627,-6.1153577846492 37.3105672777454,-6.04475036124726 37.2771976083931,-6.04063041298302 37.2060457158668,-6.01561413928283 37.1744528799079,-6.03401959555254 37.1582677514823,-6.01528760169561 37.1211642619874,-6.03587498721126 37.0508366627984,-6.08415258431292 37.0557470251154,-6.1443715598251 37.0182321851901,-6.20489081698833 36.9013662293736,-6.24848183495607 36.8846441975983,-6.28133674315331 36.9000390906331,-6.27285487885334 36.8641095189495,-6.21240564569216 36.8569441225281,-6.19273049982008 36.8273222937041,-6.23650318117893 36.7815132843624,-6.25972339425557 36.7794168824025,-6.29452828553456 36.7134050019461,-6.32211152509425 36.693679078589,-6.40354402587554 36.7535989999008,-6.39366914786855 36.7750231637474,-6.42465019950879 36.7928589511795,-6.48913510482893 36.9063367348853,-6.55574132052792 36.9640015939793,-6.54729285159512 36.9812830814621,-6.79440295589744 37.1191319871777))'),
               ('title', 'Protected Area wkt'),
               ('abstract', 'Protected Area wkt')])

In [4]:
spatialCover = dict([('id', 'spatialCover'),
               ('value', '100'),
               ('title', 'Overlap percentage between PA and dataset'),
               ('abstract', 'Overlap percentage between PA and dataset')])

In [4]:
crop_flag = dict([('id', 'crop_flag'),
               ('value', 'False'),
               ('title', 'Flag for cropping'),
               ('abstract', 'Flag for cropping (True/False)')])

In [5]:
start = dict([('id', 'start'),
              ('value', '2018-06-08T00:00:00Z'),
              ('title', 'start date for dataset selection'),
              ('abstract', 'start date for dataset selection')])

In [6]:
stop = dict([('id', 'stop'),
            ('value', '2018-06-08T23:59:59Z'),
            ('title', 'stop date for dataset selection'),
            ('abstract', 'stop date for dataset selection')])

In [7]:
cat_index = dict([('id', 'cat_index'),
              ('value', 'ecop-cnr-issia'),
              ('title', 'publishing catalog index'),
              ('abstract', 'publishing catalog index')])

In [8]:
config_url = dict([('id', 'config_url'),
                   ('value', 'https://store.terradue.com/ec-ecopotential/config/cnr-sftp-config.ini'),
                   ('title', 'FTP configuration file URL'),
                   ('abstract', 'FTP configuration file URL')])

In [9]:
shapefile_url = dict([('id', 'shapefile_url'),
                      ('value', 'https://store.terradue.com/ec-ecopotential/PAs/Shapefiles/Donana/Donana_Biosfera_LatLonWGS84'),
                      ('title', 'Protected Area shapefiles base name download URL'),
                      ('abstract', 'Protected Area name shape file download URL')])

In [10]:
username = dict([('id', 'username'),
                 ('value', 'ecop-cnr-issia'),
                 ('title', 'username for DA access'),
                 ('abstract', 'username for DA access')])

In [11]:
api_key  = dict([('id', 'api_key'),
                 ('value', 'AKCp5aUa1sSVSDdky5odFrKbwpyKzrBuZjWvrWbUigaPVNPWL3H6uiLSgEjFWLVdnUz45Ky7x'),
                 ('title', 'apikey for DA access'),
                 ('abstract', 'apikey for DA access')])

In [35]:
input_reference = "https://catalog.terradue.com/sentinel1/series/SLC_GRDH/search?geom=POLYGON((-6.79440295589744%2037.1191319871777%2C-6.80827154725944%2037.1673031942056%2C-6.72121114356795%2037.2425767108835%2C-6.71042775152577%2037.2930498879682%2C-6.73405462341917%2037.3301447063348%2C-6.71685147187095%2037.3522253716156%2C-6.67435624435345%2037.3534492963332%2C-6.64461873043217%2037.3365633914571%2C-6.62625336367183%2037.3569789562915%2C-6.59581018720718%2037.3506994424118%2C-6.58125246058411%2037.3703051383248%2C-6.492413660497%2037.3590379521576%2C-6.4716624422663%2037.3807359777278%2C-6.43859705183134%2037.368908852933%2C-6.44398095303713%2037.347088527492%2C-6.42200724935979%2037.3329929700694%2C-6.37035012694059%2037.3583637388823%2C-6.33153540861988%2037.3329986280188%2C-6.25703879672152%2037.3486795386779%2C-6.2123425985781%2037.3266608721382%2C-6.1976208676784%2037.283024780795%2C-6.17366048732846%2037.2660742144794%2C-6.15177710683581%2037.2667431439207%2C-6.14475916204647%2037.3055642119627%2C-6.1153577846492%2037.3105672777454%2C-6.04475036124726%2037.2771976083931%2C-6.04063041298302%2037.2060457158668%2C-6.01561413928283%2037.1744528799079%2C-6.03401959555254%2037.1582677514823%2C-6.01528760169561%2037.1211642619874%2C-6.03587498721126%2037.0508366627984%2C-6.08415258431292%2037.0557470251154%2C-6.1443715598251%2037.0182321851901%2C-6.20489081698833%2036.9013662293736%2C-6.24848183495607%2036.8846441975983%2C-6.28133674315331%2036.9000390906331%2C-6.27285487885334%2036.8641095189495%2C-6.21240564569216%2036.8569441225281%2C-6.19273049982008%2036.8273222937041%2C-6.23650318117893%2036.7815132843624%2C-6.25972339425557%2036.7794168824025%2C-6.29452828553456%2036.7134050019461%2C-6.32211152509425%2036.693679078589%2C-6.40354402587554%2036.7535989999008%2C-6.39366914786855%2036.7750231637474%2C-6.42465019950879%2036.7928589511795%2C-6.48913510482893%2036.9063367348853%2C-6.55574132052792%2036.9640015939793%2C-6.54729285159512%2036.9812830814621%2C-6.79440295589744%2037.1191319871777))&correlatedTo=https://catalog.terradue.com/ecop-cnr-issia/series/pa/search?uid=donana&corFunction=spatial"


SyntaxError: invalid syntax (<ipython-input-35-a5e8c5c8ab2c>, line 1)

In [13]:
ftp_server_basepath = 'sftp://frontend.recas.ba.infn.it/lustre/ecopotential/incoming/PAs'

In [14]:
data_path = '/workspace/data'

In [15]:
input_identifier = 'S1A_IW_SLC__1SDV_20180503T182621_20180503T182648_021744_025843_8690'

In [19]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import snappy
import dateutil.parser as parser
import gc
from datetime import datetime
import matplotlib
import matplotlib.colors as colors

from os.path import basename
import gdal
import osr
import warnings
warnings.filterwarnings("ignore")
import zipfile

import sys

sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())
import ISOMetadata

%matplotlib inline

In [21]:
s1prd = "%s/%s/%s.SAFE/manifest.safe" % (data_path, input_identifier, input_identifier)
product = snappy.ProductIO.readProduct(s1prd)

width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()
name = product.getName()
description = product.getDescription()
band_names = product.getBandNames()


In [22]:
product_date = parser.parse(product.getStartTime().toString()).date()

In [23]:
output_date = '%s%02d%02d' % (product_date.year, product_date.month, product_date.day)
date_path = '%s/%02d/%02d' % (product_date.year, product_date.month, product_date.day)
    
middle_path = '%s/EO_Data/Sentinel1/Raw'%pa_name['value']
        
ftp_remote_path = os.path.join(ftp_server_basepath, date_path)
zip_output_name = '%s_PA_%s_CROP.zip' % (name, pa_name['value'])
download_URL = '%s/%s' %(ftp_remote_path, zip_output_name)

### Getting Shapefile from store

In [24]:
import warnings
import requests
warnings.filterwarnings("ignore")
import shapefile
from shapely.geometry import shape

try:
    shp_url = '%s.shp' %shapefile_url['value']
    r = requests.get(shp_url,headers={"X-JFrog-Art-Api":api_key['value'], 'User-Agent': 'curl/t2Client'})
    if r.status_code == 200:
        if r.content:
            shape_shp = os.path.join(os.path.join(data_path,os.path.basename(shp_url)))
            with open(shape_shp,"wb") as f:
                for chunk in r.iter_content(chunk_size=1024):
                    # writing one chunk at a time to pdf file
                    if chunk:
                        f.write(chunk)
                f.close()
    else:
        raise ValueError
    dbf_url = '%s.dbf' %shapefile_url['value']
    r2 = requests.get(dbf_url,headers={"X-JFrog-Art-Api":api_key['value'], 'User-Agent': 'curl/t2Client'})
    if r2.status_code == 200:
        if r2.content:
            shape_dbf = os.path.join(os.path.join(data_path,os.path.basename(dbf_url)))
            with open(shape_dbf,"wb") as f:
                for chunk in r2.iter_content(chunk_size=1024):
                    # writing one chunk at a time to pdf file
                    if chunk:
                        f.write(chunk)
                f.close()
    else:
        raise ValueError
except Exception as e:
    raise

shapeIN = shapefile.Reader(shape_shp)
feature = shapeIN.shapeRecords()[0]
first = feature.shape.__geo_interface__  

shp_geom = shape(first)

### Crop

In [34]:
snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

HashMap = snappy.jpy.get_type('java.util.HashMap')

WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')
geom = WKTReader().read(str(shp_geom))

    
subsets = []
geotiffs = []


for band in list(band_names)[0:18]:

    
    BandDescriptor = snappy.jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')

    parameters = HashMap()
    parameters.put('referenceBand', band)

    resample = snappy.GPF.createProduct('Resample', parameters, product)

    parameters = HashMap()
    parameters.put('copyMetadata', True)
    parameters.put('sourceBands', band)

    subset = snappy.GPF.createProduct('Subset', parameters, resample)

    parameters = HashMap()
    parameters.put('copyMetadata', True)
    parameters.put('geoRegion', geom)

    subset_geo = snappy.GPF.createProduct('Subset', parameters, subset)

    subsets.append(subset_geo)

    output_name = '%s_PA_%s_CROP_%s.tif' % (name, pa_name['value'], band)
    geotiffs.append(output_name)

    snappy.ProductIO.writeProduct(subset_geo, output_name, 'GeoTIFF')

RuntimeError: org.esa.snap.core.gpf.OperatorException: Source product 'S1A_IW_SLC__1SDV_20180503T182621_20180503T182648_021744_025843_8690_resampled' contains rasters of different sizes and can not be processed.
Please consider resampling it so that all rasters have the same size.

In [ ]:
%matplotlib inline

def plotBand(product, band, vmin, vmax):
    
    band = product.getBand(band)

    w = band.getRasterWidth()
    h = band.getRasterHeight()
    
    band_data = np.zeros(w * h, np.float32)
    band.readPixels(0, 0, w, h, band_data)

    band_data.shape = h, w

    imgplot = plt.imshow(band_data, cmap=plt.cm.binary_r, vmin=vmin, vmax=vmax)

    
    return imgplot 


In [ ]:
if False:
    fig = plt.figure(figsize=(20,20))
    for index, subset in enumerate(subsets):
        a=fig.add_subplot(4,4,index+1)
        imgplot = plotBand(subset, band_names[index], 0, 1)
        nameB = band_names[index]
        a.set_title('%s %s-cropped' %(nameB, pa_name['value']))
    
    plt.tight_layout()
    fig = plt.gcf() 
    plt.show()

    fig.clf()
    plt.close()
    gc.collect()
    

In [ ]:
def set_dates(start_date,end_date):
    
    iso_metadata.set_start_date(start_date)
    iso_metadata.set_end_date(end_date)

In [ ]:
def set_info(short_name, title, abstract):
    
    iso_metadata.set_identifier(short_name)
    iso_metadata.set_title(title)
    iso_metadata.set_abstract(abstract)

In [ ]:
def set_geo(geotiff):
    
    ds = gdal.Open(geotiff)

    iso_metadata.set_col_size(str(ds.RasterXSize))
    iso_metadata.set_row_size(str(ds.RasterYSize))

    transform = ds.GetGeoTransform()

    iso_metadata.set_pixel_size(str(transform[1]))

    ul_x = transform[0]
    ul_y = transform[3]

    nw_corner = '%s %s' % (str(ul_x), str(ul_y))

    iso_metadata.set_nw_corner(nw_corner)

    lr_x = transform[0] + transform[1] * ds.RasterXSize
    lr_y = transform[3] + transform[5] * ds.RasterYSize

    se_corner = '%s %s' % (str(lr_x), str(lr_y))

    iso_metadata.set_se_corner(se_corner)

    
    
    old_cs= osr.SpatialReference()
    old_cs.ImportFromWkt(ds.GetProjectionRef())

    wgs84_wkt = """
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.01745329251994328,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]"""
    new_cs = osr.SpatialReference()
    new_cs .ImportFromWkt(wgs84_wkt)

    transform = osr.CoordinateTransformation(old_cs,new_cs) 

    min_lon = transform.TransformPoint(ul_x,lr_y)[0]
    iso_metadata.set_min_lon(str(min_lon))

    min_lat = transform.TransformPoint(ul_x,lr_y)[1]
    iso_metadata.set_min_lat(str(min_lat))

    max_lon = transform.TransformPoint(lr_x, ul_y)[0]
    iso_metadata.set_max_lon(str(max_lon))

    max_lat = transform.TransformPoint(lr_x, ul_y)[1]
    iso_metadata.set_max_lat(str(max_lat))
    
    prj = ds.GetProjection()
    srs=osr.SpatialReference(wkt=prj)


    iso_metadata.set_epsg_code(srs.GetAttrValue("PROJCS|AUTHORITY", 1))

In [ ]:
def write_properties(properties_file):
    
    properties = open(properties_file + '.properties', 'w')

    properties.write('identifier=' + geotiff)
    properties.write('\n')
    properties.write('date=%s/%s' % (parser.parse(subset.getStartTime().toString()).isoformat(), 
                                 parser.parse(subset.getEndTime().toString()).isoformat()))
    properties.write('\n')
    properties.write('category=%s,http://www.terradue.com/api/data-pipeline/results,%s Protected Area resource' % (pa_code['value'], pa_name['value']))
 
    properties.close()

In [ ]:
iso_metadata = ISOMetadata.ISOMetadata()
iso_metadata.set_contact('info@terradue.com')
iso_metadata.set_date(output_date)
iso_metadata.set_data_format('GEOTIFF')
iso_metadata.set_data_type('UInt16')
iso_metadata.set_pa(pa_name['value'])
iso_metadata.set_data_quality('Nominal')
iso_metadata.set_responsible_party('CNR')
iso_metadata.set_onlineResource(download_URL)

In [ ]:
for index, geotiff in enumerate(geotiffs):
        
    subset = subsets[index]
        
    start_date = parser.parse(subset.getStartTime().toString()).isoformat()
    end_date = parser.parse(subset.getEndTime().toString()).isoformat()
        
    set_dates(start_date,end_date)
        
    short_name = os.path.splitext(basename(geotiff))[0]
    band = band_names[index]
    title = 'Sentinel 1 Level 1 %s Band %s' % (pt,band)
    abstract = 'Sentinel 1 Level 1 %s cropped Band %s' % (pt,band)
        
    set_info(short_name, title, abstract)

    set_geo(geotiff)

    iso_metadata.write(os.path.splitext(basename(geotiff))[0] + '.xml')

    write_properties(basename(geotiff))

### Zipping the cropped bands and related metadata

In [ ]:

zip_output_name = '%s_PA_%s_CROP.zip' % (name, pa_name['value'])
zf = zipfile.ZipFile(zip_output_name, mode='w')
try:
    for geotiff in geotiffs:
        short_name = os.path.splitext(basename(geotiff))[0]
        #print '%s %s.properties %s.xml' %(geotiff,geotiff,short_name)
        zf.write(geotiff)
        zf.write('%s.properties' %geotiff)
        zf.write('%s.xml' %short_name)
finally:
    #print 'closing'
    zf.close()

### Writing General XML

In [ ]:
set_dates(start_date,end_date)
    
short_name = os.path.splitext(zip_output_name)[0]
title = 'Sentinel 1 Level 1 %s (cropped Bands) ' % name
abstract = 'Sentinel 1 Level 1 cropped over %s \(17 bands\)' % pa_name['value']
        
set_info(short_name, title, abstract)

iso_metadata.write(os.path.splitext(zip_output_name)[0] + '.xml')